In [1]:
import numpy as np
import pandas as pd

import os
import random
import shutil
import sys

from collections import defaultdict
from sklearn.preprocessing import LabelEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchmetrics

import lightning as L
from lightning.pytorch.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

In [2]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
from utilities.data_downloader import train_val_test_downloader
from utilities.upsampling import upsampling
from utilities.plots import plt, COLORMAP, visualize_latent

In [4]:
from warnings import simplefilter
simplefilter("ignore", category=RuntimeWarning)

In [5]:
def set_random_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    np.random.seed(seed)
    random.seed(seed)

set_random_seed(42)
L.seed_everything(42)

Seed set to 42


42

# Description

bla bla

# Dataset
bla bla

In [6]:
train, val, test, labels = train_val_test_downloader('interp')

Datasets downloaded
 - train  : 810 entries
 - val    : 174 entries
 - test   : 174 entries
 - labels : 1158 entries


In [7]:
train_upsampled = upsampling(train)
len(train_upsampled)

81000

In [8]:
class LightCurveDataset(Dataset):
    def __init__(self, dataframe:pd.DataFrame,
                 data_col:str='lgRate',
                 weight_col:str='weight'):
        
        data = np.array(dataframe.loc[:, data_col].tolist(),
                        dtype=np.float32)
        weight = np.array(dataframe.loc[:, weight_col].tolist(),
                          dtype=np.float32)
        
        self.data = torch.from_numpy(
            data).unsqueeze(dim=1)   # value
        self.weight = torch.from_numpy(
            weight).unsqueeze(dim=1) # weight

        # using dataframe index = event names 
        # as labels
        labels = dataframe.index
        self.label_enc = LabelEncoder()
        self.labels = torch.as_tensor(self.label_enc.fit_transform(labels))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx], self.weight[idx]

In [9]:
train_dataset, val_dataset, test_dataset = (
    LightCurveDataset(train_upsampled),
    LightCurveDataset(val),
    LightCurveDataset(test)
)

In [10]:
train_loader = DataLoader(train_dataset,
                          batch_size=32,
                          shuffle=True,
                          num_workers=0)
val_loader = DataLoader(val_dataset,
                        batch_size=32,
                        shuffle=False,
                        num_workers=0)
test_loader = DataLoader(test_dataset,
                         batch_size=32,
                         shuffle=False,
                         num_workers=0)

# for predictions on non-augmented train:
train_loader_ = DataLoader(LightCurveDataset(train),
                           batch_size=32,
                           shuffle=False,
                           num_workers=0)

# Model
bla bla

## Torch Models

In [11]:
class Encoder(nn.Module):
    def __init__(self, latent_dim:int,
                 architecture:tuple=(32, 4),
                 tseries_length:int=64):
        super().__init__()

        self.hidden_dims = [
            architecture[0]* 2**pow for pow in range(architecture[1])
            ]                                       # num of filters in layers
        self.tseries_length = tseries_length

        modules = []
        in_channels = 1                             # initial num of channels
        for h_dim in self.hidden_dims:              # conv layers
            modules.append(
                nn.Sequential(
                    nn.Conv1d(
                        in_channels=in_channels,    # num of input channels
                        out_channels=h_dim,         # num of output channels
                        kernel_size=3,
                        stride=2,                   # convolution kernel step
                        padding=1,                  # save shape
                    ),
                    nn.BatchNorm1d(h_dim),
                    nn.LeakyReLU(),
                )
            )
            in_channels = h_dim                     # changing num of 
                                                    # input channels for 
                                                    # next iteration

        modules.append(nn.Flatten())                # to vector
        intermediate_dim = (
            self.hidden_dims[-1] * 
            self.tseries_length // (2**len(self.hidden_dims))
        )
        modules.append(nn.Linear(in_features=intermediate_dim,
                                 out_features=latent_dim))

        self.encoder = nn.Sequential(*modules)

    def forward(self, x):
        x = self.encoder(x)
        return x


class Decoder(nn.Module):
    def __init__(self, latent_dim:int,
                 architecture:tuple=(32, 4),
                 tseries_length:int=64):
        super().__init__()
        self.hidden_dims = [
            architecture[0]* 2**pow for pow in range(architecture[1]-1, 0, -1)
            ]                                       # num of filters in layers
        self.tseries_length = tseries_length

        intermediate_dim = (
            self.hidden_dims[0] * 
            self.tseries_length // (2**len(self.hidden_dims))
        )
        self.linear = nn.Linear(in_features=latent_dim,
                                out_features=intermediate_dim)

        modules = []
        for i in range(len(self.hidden_dims) - 1):  # define upsample layers
            modules.append(
                nn.Sequential(
                    nn.Upsample(scale_factor=2),
                    nn.Conv1d(
                        in_channels=self.hidden_dims[i],
                        out_channels=self.hidden_dims[i + 1],
                        kernel_size=3,
                        padding=1,
                    ),
                    nn.BatchNorm1d(self.hidden_dims[i + 1]),
                    nn.LeakyReLU(),
                )
            )

        modules.append(
            nn.Sequential(
                nn.Upsample(scale_factor=2),
                nn.Conv1d(in_channels=self.hidden_dims[-1],
                          out_channels=1,
                          kernel_size=3, padding=1)
            )
        )

        self.decoder = nn.Sequential(*modules)

    def forward(self, x):
        x = self.linear(x)        # from latents space to Linear
        x = x.view(
            -1, self.hidden_dims[0],
            self.tseries_length // (2**len(self.hidden_dims))
            )                     # reshape
        x = self.decoder(x)       # reconstruction
        return x

class VAEncoder(Encoder):
    def __init__(self, latent_dim):
        if latent_dim % 2 != 0:   # check for the parity of the latent space
            raise Exception('Latent size for VAEncoder must be even')

        super().__init__(latent_dim)

## Lightning wrapper

In [12]:
class LitAE(L.LightningModule):
    def __init__(self, encoder, decoder, derivative_weight=1.0):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.derivative_weight = derivative_weight

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=1e-4)

    def forward_handler(self, data,
                        *args, **kwargs):
        # here is the logic how data is moved through AE
        latent = self.encoder(data)
        recon = self.decoder(latent)
        return latent, recon

    def loss_handler(self, recon, data, weight, latent,
                     *args, **kwargs):
        # here is the loss function computing
        recon_loss = torch.masked_select(
            input = F.mse_loss(
                recon, data, reduction='none'
            ) * weight,
            mask = weight.ge(0.0)
        )
        recon_loss = recon_loss.mean()

        # derivative penalty = 
        # L1-regularization of the output timeseries
        derivative_loss = torch.abs(
            torch.diff(recon, dim=-1)
        ).mean()

        # total loss
        loss = recon_loss + self.derivative_weight * derivative_loss

        return loss

    def training_step(self, batch, batch_idx):
        data, labels, weight = batch

        latent, recon = self.forward_handler(data, labels)
        loss = self.loss_handler(recon, data, weight, latent)

        self.log('train_loss', loss, on_step=False, on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        data, labels, weight = batch

        latent, recon = self.forward_handler(data, labels)
        loss = self.loss_handler(recon, data, weight, latent)

        self.log('val_loss', loss, on_step=False, on_epoch=True)
        return loss

    def on_test_epoch_start(self):
        # create dict with empty tensors for further accumulating over batches
        self.test_result = defaultdict(torch.Tensor)

    def test_step(self, batch, batch_idx):
        data, labels, weight = batch

        latent, recon = self.forward_handler(data, labels)
        self.update_test_result(data, weight, recon, latent, labels)

    def update_test_result(self, data, weight, recon, latent, labels):
        # accumulating results every batch
        self.test_result['real'] = torch.cat(
            [self.test_result['real'], data.cpu()]
        )
        self.test_result['weight'] = torch.cat(
            [self.test_result['weight'], weight.cpu()]
        )
        self.test_result['recon'] = torch.cat(
            [self.test_result['recon'], recon.cpu()]
        )
        self.test_result['latent'] = torch.cat(
            [self.test_result['latent'], latent.cpu()]
        )
        self.test_result['labels'] = torch.cat(
            [self.test_result['labels'], labels.cpu()]
        )

    def on_test_epoch_end(self):
        # simply change type from torch tensor to numpy array
        # for every item in test_result dictionary
        for key in self.test_result:
            self.test_result[key] = self.test_result[key].numpy()

In [13]:
class LitVAE(LitAE):
    def __init__(self, encoder, decoder,
                 derivative_weight=1.0,
                 kld_weight=0.005,
                 ):
        super().__init__(encoder, decoder, derivative_weight)
        self.kld_weight = kld_weight

    def vae_split(self, latent):
        size = (
            latent.shape[1] // 2
        )  # divide the latent representation into mu and log_var
        mu = latent[:, :size]
        log_var = latent[:, size:]
        return mu, log_var

    def vae_reparametrize(self, mu, log_var):
        sigma = torch.exp(0.5 * log_var)
        eps = torch.randn(mu.shape[0], mu.shape[1]).to(self.device)
        return eps * sigma + mu

    def kld_loss(self, mu, log_var):
        var = log_var.exp()
        kl_loss = torch.mean(
            -0.5 * torch.sum(log_var - var - mu**2 + 1, dim=1), dim=0
        )
        return kl_loss

    def forward_handler(self, data, *args, **kwargs):
        # here is the logic how data is moved through AE
        latent = self.encoder(data)

        mu, log_var = self.vae_split(latent)
        sample = self.vae_reparametrize(mu, log_var)

        recon = self.decoder(sample)
        return latent, recon

    def loss_handler(self, recon, data, weight, latent, *args, **kwargs):
        mu, log_var = self.vae_split(latent)
        # here is the loss function computing
        loss = torch.masked_select(
            input = F.mse_loss(recon, data, reduction='none') * weight,
            mask = weight.ge(0.0)).mean() + self.derivative_weight * torch.abs(
            torch.diff(recon, dim=-1)
            ).mean() + self.kld_weight * self.kld_loss(mu, log_var)
        return loss

## Utilities

In [14]:
def reparametrize_latent(vae, latent):
    mu, log_var = vae.vae_split(latent)
    var = np.exp(log_var)

    mu, log_var = torch.tensor(mu), torch.tensor(log_var)
    sample = vae.vae_reparametrize(mu, log_var).numpy()
    return sample

In [15]:
def get_dict_result(trainer, model, dataloader, ckpt_path):
    
    with torch.no_grad():
        trainer.test(model, dataloader, ckpt_path=ckpt_path)
    model.test_result[
        'labels'
    ]=dataloader.dataset.label_enc.inverse_transform(
    model.test_result[
        'labels'
        ].astype(int)
    )

    real = model.test_result['real'].squeeze()
    recon = model.test_result['recon'].squeeze()
    weight = model.test_result['weight'].squeeze()


    weightedMSE = (real-recon)**2 * weight
    pred_errors = (weightedMSE ** 0.5).tolist()

    weightedMSE = np.ma.masked_array(data=weightedMSE,
                                     mask=~(weight.astype(bool))
    )
    weightedMSE = weightedMSE.mean(axis=1, keepdims=True)

    latent = model.test_result['latent'].copy()

    if hasattr(model, 'vae_reparametrize') and callable(model.vae_reparametrize):
        # for VAE, we must reparametrize latent first
        latent = reparametrize_latent(model, latent)

    latentdim = latent.shape[-1]

    latent = pd.DataFrame(
        data=np.concatenate((latent, weightedMSE), axis=1),
        index=model.test_result['labels'],
        columns=['feature_'+str(dim) for dim in range(latentdim)]+['wMSE'])

    latent.insert(loc=latent_dim+1, column='pred_error', value=pred_errors)

    return latent, real, recon, weight

# Training

In [16]:
latent_dim = 3

encoder, decoder = Encoder(latent_dim), Decoder(latent_dim)

In [17]:
encoder

Encoder(
  (encoder): Sequential(
    (0): Sequential(
      (0): Conv1d(1, 32, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Conv1d(32, 64, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Conv1d(64, 128, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (3): Sequential(
      (0): Conv1d(128, 256, kernel_size=(3,), stride=(2,), padding=(1,))
      (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): LeakyReLU(negative_slope=0.01)
    )
    (4): Flatten(start_dim=1, end_dim=

In [18]:
decoder

Decoder(
  (linear): Linear(in_features=3, out_features=2048, bias=True)
  (decoder): Sequential(
    (0): Sequential(
      (0): Upsample(scale_factor=2.0, mode='nearest')
      (1): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
    (1): Sequential(
      (0): Upsample(scale_factor=2.0, mode='nearest')
      (1): Conv1d(128, 64, kernel_size=(3,), stride=(1,), padding=(1,))
      (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): LeakyReLU(negative_slope=0.01)
    )
    (2): Sequential(
      (0): Upsample(scale_factor=2.0, mode='nearest')
      (1): Conv1d(64, 1, kernel_size=(3,), stride=(1,), padding=(1,))
    )
  )
)

In [19]:
!rm -rf lightning_logs
!ls

AutoEncoder.ipynb requirements.txt


In [20]:
!mkdir lightning_logs
!mkdir lightning_logs/AE_latent_dim={latent_dim}

In [21]:
%load_ext tensorboard
%tensorboard --logdir lightning_logs/AE_latent_dim={latent_dim} --port 6010

Reusing TensorBoard on port 6010 (pid 99622), started 1:09:01 ago. (Use '!kill 99622' to kill it.)

In [22]:
autoencoder = LitAE(encoder, decoder)

exp_name = f'AE_latent_dim={latent_dim}'

checkpoint_callback = ModelCheckpoint(
    dirpath=f'./lightning_logs/AE_latent_dim={latent_dim}',
    filename='best', monitor='val_loss', mode='min')

early_stopping_callback = EarlyStopping(
    monitor='val_loss', mode='min', min_delta=0, patience=50)

logger = TensorBoardLogger(save_dir='./lightning_logs', name=exp_name)

trainer = L.Trainer(max_epochs=500, logger=logger,
                    callbacks=[checkpoint_callback, early_stopping_callback])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(autoencoder, train_loader, val_loader)

/Users/nickolaymartynenko/anaconda3/lib/python3.11/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:654: Checkpoint directory /Users/nickolaymartynenko/Downloads/GRB-X-Ray-Afterglow/models/AutoEncoder/lightning_logs/AE_latent_dim=3 exists and is not empty.

  | Name    | Type    | Params | Mode 
--------------------------------------------
0 | encoder | Encoder | 133 K  | train
1 | decoder | Decoder | 131 K  | train
--------------------------------------------
265 K     Trainable params
0         Non-trainable params
265 K     Total params
1.062     Total estimated model params size (MB)
36        Modules in train mode
0         Modules in eval mode


Sanity Checking: |                                        | 0/? [00:00<?, ?it/s]

In [2]:
import torch
torch.cuda.is_available()

False